In [307]:
import numpy as np 
import math
import warnings
warnings.filterwarnings("ignore")


In [308]:
class Activition_functions():

    @staticmethod
    def Tanh(z):
        return (math.exp(z) - math.exp(-z)) / (math.exp(z) + math.exp(-z))
     
    def softmax(vector):
        e = np.exp(vector)
        return e / e.sum()

In [309]:
W_H = np.array([[0.1 ,0.5 , 0.1],[0.5,0.9,0.2],[0.1,0.3,0.1]])

W_X = np.array([[0.6,0.8,0.4,0.8],[0.2,0.2,0.8,0.7],[0.9,0.8,0.1,0.2]])

W_Y = np.array([[0.9,0.8,0.3],[0.2,0.3,0.4],[0.6,0.9,0.1],[0.5,0.0,0.3]])

v = np.dot(W_H,np.zeros((3,1))) + np.dot(W_X,np.array([1,0,0,0]).reshape((4,1)))

c = np.array([Activition_functions.Tanh(float(x))  for x in list(v)]).reshape((3,1))


Activition_functions.softmax(np.dot(W_Y,c))

array([[0.321599  ],
       [0.2149301 ],
       [0.24193486],
       [0.22153605]])

In [ ]:
class RNN():
    def __init__(self,word:str,k:int,d:int,W_H: np.array = None, W_X: np.array = None, W_Y: np.array = None):
        self.k =k
        self.d =d
        self.word = word
        self.W_H = np.random.uniform(-1, 1, size=(self.d, self.d)) if W_H is None else W_H
        self.W_X = np.random.uniform(-1, 1, size=(self.d, self.k)) if W_X is None else W_X
        self.W_Y = np.random.uniform(-1, 1, size=(self.k, self.d)) if W_Y is None else W_Y
        self.h = {"t1":np.zeros((self.d,1))}
        self.letter = self.__one_hot_list_Char() if len(self.word.split(" ")) == 1 else self.__one_hot_list_word()
        self.error = 0
        self.output = 0


    def __one_hot_list_Char(self):
        unique_chars = self.word
        char_to_index = {char: idx for idx, char in enumerate(unique_chars)}
        one_hot_vectors = []
        for char in self.word:
            vector = [0] * len(unique_chars)  
            vector[char_to_index[char]] = 1 
            one_hot_vectors.append(vector)    
        return one_hot_vectors

    def __one_hot_list_word(self):
        unique_chars = self.word.split(" ")
        char_to_index = {char: idx for idx, char in zip(list(range(len(unique_chars))),unique_chars)}
        one_hot_vectors = []
        for char in self.word.split(" "):
            vector = [0] * len(unique_chars)  
            vector[char_to_index[char]] = 1 
            one_hot_vectors.append(vector)   
        return one_hot_vectors
    
    
    def Rnn_forward(self):
        for x in range(len(self.letter)-1):
            first_step = np.dot(self.W_H,self.h[f"t{x+1}"]) + np.dot(self.W_X,np.array(self.letter[x]).reshape((4,1)))
            second_step = np.array([Activition_functions.Tanh(float(x))  for x in list(first_step)]).reshape((3,1))
            self.h[f"t{x+1+1}"] = second_step
            result =  Activition_functions.softmax(np.dot(self.W_Y,second_step))
        self.error = self.mean_squared_error(result,self.letter[-1])
        self.output = result
            
        return result

    def mean_squared_error(self,y_pred, y_true):
        shape = np.array(y_pred).shape
        return  (np.array(y_pred) - np.array(y_true).reshape(shape))**2 

In [383]:
rnn = RNN("I am Youssef Mustafa",k=4,d=3,W_H=None,W_X=None,W_Y=None)
last_word  = rnn.Rnn_forward()
last_word




array([[0.31714663],
       [0.37573836],
       [0.13593551],
       [0.1711795 ]])

In [377]:
import numpy as np

class RNN():
    def __init__(self, word: str, k: int, d: int, W_H: np.array = None, W_X: np.array = None, W_Y: np.array = None):
        self.k = k  
        self.d = d  
        self.word = word
        
        self.W_H = np.random.uniform(-1, 1, size=(self.d, self.d)) if W_H is None else W_H
        self.W_X = np.random.uniform(-1, 1, size=(self.d, self.k)) if W_X is None else W_X
        self.W_Y = np.random.uniform(-1, 1, size=(self.k, self.d)) if W_Y is None else W_Y
       
        self.h = {"t0": np.zeros((self.d, 1))}  
        self.letter = self.__one_hot_list_Char() if len(self.word.split(" ")) == 1 else self.__one_hot_list_word()
        self.cache = {}  
        self.loss_history = []  

    def __one_hot_list_Char(self):
        unique_chars = sorted(list(set(self.word)))
        char_to_index = {char: idx for idx, char in enumerate(unique_chars)}
        one_hot_vectors = []
        for char in self.word:
            vector = np.zeros(len(unique_chars))
            vector[char_to_index[char]] = 1
            one_hot_vectors.append(vector)
        return one_hot_vectors

    def __one_hot_list_word(self):
        unique_words = self.word.split(" ")
        word_to_index = {word: idx for idx, word in enumerate(unique_words)}
        one_hot_vectors = []
        for word in unique_words:
            vector = np.zeros(len(unique_words))
            vector[word_to_index[word]] = 1
            one_hot_vectors.append(vector)
        return one_hot_vectors

    def forward(self):
        self.cache = {}
        self.h = {"t0": np.zeros((self.d, 1))}  
        for t in range(len(self.letter) - 1):
            x_t = np.array(self.letter[t]).reshape((-1, 1)) 
            h_prev = self.h[f"t{t}"] 
            z_t = np.dot(self.W_H, h_prev) + np.dot(self.W_X, x_t)  
            h_t = np.tanh(z_t)  
            self.cache[f"t{t+1}"] = (x_t, h_prev, z_t)
            self.h[f"t{t+1}"] = h_t
        

        y_pred = self.W_Y.dot(self.h[f"t{len(self.letter)-1}"])  
        y_pred_softmax = Activition_functions.softmax(y_pred)
        loss = self.mean_squared_error(y_pred_softmax, np.array(self.letter[-1]).reshape((-1, 1)))
        return y_pred_softmax, loss

    def backward(self, learning_rate=0.01):
        T = len(self.letter) - 1
        dW_H = np.zeros_like(self.W_H)
        dW_X = np.zeros_like(self.W_X)
        dW_Y = np.zeros_like(self.W_Y)
        dh_next = np.zeros_like(self.h["t0"])
        
        
        y_true = np.array(self.letter[-1]).reshape((-1, 1))
        y_pred, _ = self.forward()
        dy_pred = y_pred - y_true  
        
        
        dW_Y = dy_pred.dot(self.h[f"t{T}"].T)  
        
        
        for t in reversed(range(1, T + 1)):
            x_t, h_prev, z_t = self.cache[f"t{t}"]
            dh_t = self.W_Y.T.dot(dy_pred) + dh_next 
            dz_t = dh_t * (1 - np.tanh(z_t) ** 2)  
            dW_H += dz_t.dot(h_prev.T)
            dW_X += dz_t.dot(x_t.T)
            dh_next = self.W_H.T.dot(dz_t)
        
        
        self.W_H -= learning_rate * dW_H
        self.W_X -= learning_rate * dW_X
        self.W_Y -= learning_rate * dW_Y
        return dW_H, dW_X, dW_Y

    def train(self, epochs=100, learning_rate=0.01, verbose=True):
        for epoch in range(epochs):
            prediction, loss = self.forward()
            self.backward(learning_rate)
            self.loss_history.append(loss)
            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}, Loss: {np.mean(loss):.4f}")
        return prediction
    

    def mean_squared_error(self,y_pred, y_true):
        shape = np.array(y_pred).shape
        return  (np.array(y_pred) - np.array(y_true).reshape(shape))**2 

   

    

In [379]:

rnn = RNN(word="I am Youssef Mustafa", k=4, d=3) 

y_pred, _ = rnn.forward()
print("Final prediction after Forward pass:", y_pred)

final_output = rnn.train(epochs=200, learning_rate=0.01)
print("Final prediction after Backpropagation pass:",final_output)



Final prediction after Forward pass: [[0.32970587]
 [0.29658534]
 [0.11240122]
 [0.26130757]]
Epoch 0, Loss: 0.1887
Epoch 10, Loss: 0.1507
Epoch 20, Loss: 0.1257
Epoch 30, Loss: 0.1073
Epoch 40, Loss: 0.0926
Epoch 50, Loss: 0.0804
Epoch 60, Loss: 0.0702
Epoch 70, Loss: 0.0614
Epoch 80, Loss: 0.0538
Epoch 90, Loss: 0.0473
Epoch 100, Loss: 0.0417
Epoch 110, Loss: 0.0368
Epoch 120, Loss: 0.0325
Epoch 130, Loss: 0.0288
Epoch 140, Loss: 0.0256
Epoch 150, Loss: 0.0229
Epoch 160, Loss: 0.0204
Epoch 170, Loss: 0.0183
Epoch 180, Loss: 0.0165
Epoch 190, Loss: 0.0148
Epoch 199, Loss: 0.0136
Final prediction after Backpropagation pass: [[0.10571448]
 [0.05900639]
 [0.03164061]
 [0.80363851]]
